# Demo - Performance Comparison (CIFAR10)

## 1. Load CIAFR10

In [1]:
# https://github.com/RobustBench/robustbench
from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from utils import l2_distance

images, labels = load_cifar10(n_examples=50)
device = "cuda"

Files already downloaded and verified


## 2. Standard Accuracy

In [2]:
model_list = ['Wong2020Fast', 'Rice2020Overfitting', 'Carmon2019Unlabeled']
for model_name in model_list:
    model = load_model(model_name, norm='Linf').to(device)
    acc = clean_accuracy(model, images.to(device), labels.to(device))
    print('Model: {}'.format(model_name))
    print('- Standard Acc: {}'.format(acc))

Model: Wong2020Fast
- Standard Acc: 0.92
Model: Rice2020Overfitting
- Standard Acc: 0.86
Model: Carmon2019Unlabeled
- Standard Acc: 0.92


## 3. Torchattacks, Foolbox and ART

In [3]:
import datetime
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

# https://github.com/Harry24k/adversarial-attacks-pytorch
import torchattacks
print("torchattacks %s"%(torchattacks.__version__))

# https://github.com/bethgelab/foolbox
import foolbox as fb
print("foolbox %s"%(fb.__version__))

# https://github.com/IBM/adversarial-robustness-toolbox
import art
import art.attacks.evasion as evasion
from art.classifiers import PyTorchClassifier
print("art %s"%(art.__version__))

torchattacks 2.12.2
foolbox 3.0.0
art 1.2.0


## 3.1. Linf

### FGSM

In [4]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.FGSM(model, eps=8/255)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfFastGradientAttack(random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.FastGradientMethod(norm=np.inf, batch_size=50,
                                     classifier=classifier, eps=8/255)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.48 (15 ms)
- Foolbox
- Robust Acc: 0.48 (34 ms)
- ART
- Robust Acc: 0.48 (50 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.62 (94 ms)
- Foolbox
- Robust Acc: 0.62 (46 ms)
- ART
- Robust Acc: 0.62 (799 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.68 (18 ms)
- Foolbox
- Robust Acc: 0.68 (27 ms)
- ART
- Robust Acc: 0.68 (221 ms)



### BIM

In [5]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.BIM(model, eps=8/255, alpha=2/255, steps=10)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfBasicIterativeAttack(abs_stepsize=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.BasicIterativeMethod(batch_size=50,
                                       classifier=classifier, eps=8/255,
                                       eps_step=2/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.44 (182 ms)
- Foolbox
- Robust Acc: 0.44 (255 ms)
- ART
- Robust Acc: 0.44 (509 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.58 (2951 ms)
- Foolbox
- Robust Acc: 0.58 (3538 ms)
- ART
- Robust Acc: 0.58 (4965 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.58 (771 ms)
- Foolbox
- Robust Acc: 0.58 (930 ms)
- ART
- Robust Acc: 0.58 (1442 ms)



### PGD

In [6]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.LinfPGD(abs_stepsize=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=8/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           classifier=classifier, eps=8/255,
                                           eps_step=2/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.44 (156 ms)
- Foolbox
- Robust Acc: 0.44 (249 ms)
- ART
- Robust Acc: 0.44 (495 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.58 (2770 ms)
- Foolbox
- Robust Acc: 0.58 (3452 ms)
- ART
- Robust Acc: 0.58 (4782 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.58 (692 ms)
- Foolbox
- Robust Acc: 0.58 (945 ms)
- ART
- Robust Acc: 0.58 (1444 ms)



## 3.2. L2

### CW

In [7]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = torchattacks.CW(model, c=1, kappa=0, steps=100, lr=0.01)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2CarliniWagnerAttack(binary_search_steps=1, initial_const=1,
                                           confidence=0, steps=100, stepsize=0.01)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda'), labels.to('cuda'), epsilons=1)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))

    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.CarliniL2Method(batch_size=50, classifier=classifier, 
                                  binary_search_steps=1, initial_const=1,
                                  confidence=0, max_iter=100,
                                  learning_rate=0.01)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.14 / L2: 0.61 (4390 ms)
- Foolbox
- Robust Acc: 0.32 / L2: 0.41 (4559 ms)
- ART
- Robust Acc: 0.24 / L2: 0.71 (73368 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.22 / L2: 0.56 (44799 ms)
- Foolbox
- Robust Acc: 0.34 / L2: 0.43 (45889 ms)
- ART
- Robust Acc: 0.26 / L2: 0.65 (708854 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.26 / L2: 0.48 (13269 ms)
- Foolbox
- Robust Acc: 0.32 / L2: 0.42 (13348 ms)
- ART
- Robust Acc: 0.26 / L2: 0.62 (206416 ms)



### PGD L2

In [8]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').cuda()
    
    print("- Torchattacks")
    atk = torchattacks.PGDL2(model, eps=128/255, alpha=15/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- Foolbox")
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    atk = fb.attacks.L2PGD(abs_stepsize=15/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    _, adv_images, _ = atk(fmodel, images.to('cuda:0'), labels.to('cuda:0'), epsilons=128/255)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print("- ART")
    classifier = PyTorchClassifier(model=model, clip_values=(0, 1),
                                   loss=nn.CrossEntropyLoss(),
                                   optimizer=optim.Adam(model.parameters(), lr=0.01),
                                   input_shape=(3, 32, 32), nb_classes=10)
    atk = evasion.ProjectedGradientDescent(batch_size=50, num_random_init=0,
                                           norm = 2, classifier=classifier, eps=128/255,
                                           eps_step=15/255, max_iter=10)
    start = datetime.datetime.now()
    adv_images = torch.tensor(atk.generate(images, labels)).to(device)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} / L2: {:1.2} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

Model: Wong2020Fast
- Torchattacks
- Robust Acc: 0.68 / L2: 0.5 (166 ms)
- Foolbox
- Robust Acc: 0.68 / L2: 0.5 (267 ms)
- ART
- Robust Acc: 0.68 / L2: 0.5 (470 ms)

Model: Rice2020Overfitting
- Torchattacks
- Robust Acc: 0.7 / L2: 0.5 (2796 ms)
- Foolbox
- Robust Acc: 0.7 / L2: 0.5 (3501 ms)
- ART
- Robust Acc: 0.7 / L2: 0.5 (4822 ms)

Model: Carmon2019Unlabeled
- Torchattacks
- Robust Acc: 0.68 / L2: 0.5 (712 ms)
- Foolbox
- Robust Acc: 0.68 / L2: 0.5 (962 ms)
- ART
- Robust Acc: 0.68 / L2: 0.5 (1441 ms)

